In [5]:
import pandas as pd
import requests
import numpy as np
from flask import Flask, request, jsonify, render_template
import tensorflow as tf
import skimage
import math
import json
from pandas.io.json import json_normalize
import os

Importing traffic signs API from Nationaal Dataportaal Wegverkeer 

In [6]:
ndw = requests.get("https://data.ndw.nu/api/rest/static-road-data/traffic-signs/v1/current-state", verify=False)
print("ndw:", ndw.status_code)

C:\Users\hp\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.ndw.nu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


ndw: 200


Putting in a dataframe and data wrangling

In [7]:
signs = ndw.json()

In [8]:
signs_df = pd.DataFrame(signs)

In [9]:
signs_df

,id,type,schema_version,validated,validated_on,user_id,organisation_id,rvv_code,text_signs,location,details,publication_timestamp
0,12583107,updated,1.0,n,None,NaN,NaN,H1,[],"{'wgs84': {'latitude': '52.060835106465', 'lon...",{'image': 'https://signimages.ipsm.nl/513/5131...,2021-05-10T09:53:41.227Z
1,12583108,updated,1.0,n,None,NaN,NaN,onbekend,[],"{'wgs84': {'latitude': '52.070451883064', 'lon...",{'image': 'https://signimages.ipsm.nl/bbb/bbb2...,2021-05-10T09:53:47.701Z
2,12583109,updated,1.0,n,None,NaN,NaN,onbekend,[],"{'wgs84': {'latitude': '52.069110531122', 'lon...",{'image': 'https://signimages.ipsm.nl/918/9181...,2021-05-10T09:53:47.990Z
3,12583114,updated,1.0,n,None,NaN,NaN,onbekend,[],"{'wgs84': {'latitude': '52.062570442803', 'lon...",{'image': 'https://signimages.ipsm.nl/091/091e...,2021-05-10T10:00:02.848Z
4,12583117,updated,1.0,n,None,NaN,NaN,onbekend,[],"{'wgs84': {'latitude': '52.065265808473', 'lon...",{'image': 'https://signimages.ipsm.nl/749/749c...,2021-05-10T10:00:02.791Z
...,...,...,...,...,...,...,...,...,...,...,...,...
1858917,10059618,updated,1.0,n,None,NaN,NaN,G11,[],"{'wgs84': {'latitude': '51.806614127533', 'lon...",{'image': 'https://signimages.ipsm.nl/537/537b...,2021-09-29T10:46:11.733Z
1858918,10059621,updated,1.0,n,None,NaN,NaN,G11,[],"{'wgs84': {'latitude': '51.805638845441', 'lon...",{'image': 'https://signimages.ipsm.nl/24f/24f9...,2021-09-29T10:46:13.332Z
1858919,10059620,updated,1.0,n,None,NaN,NaN,G11,[],"{'wgs84': {'latitude': '51.808494221249', 'lon...",{'image': 'https://signimages.ipsm.nl/440/4404...,2021-09-29T10:46:09.948Z
1858920,10059623,updated,1.0,n,None,NaN,NaN,G11,[],"{'wgs84': {'latitude': '51.80331151899', 'long...",{'image': 'https://signimages.ipsm.nl/f84/f843...,2021-09-29T10:46:10.993Z


In [10]:
url = json_normalize(signs_df['details'])

<ipython-input-10-0411ea5061e1>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  url = json_normalize(signs_df['details'])


In [11]:
signs_df['image'] = url['image']

In [12]:
img = signs_df[['rvv_code','image']]

In [13]:
img

,rvv_code,image
0,H1,https://signimages.ipsm.nl/513/5131f682-f69b-4...
1,onbekend,https://signimages.ipsm.nl/bbb/bbb282fa-3822-4...
2,onbekend,https://signimages.ipsm.nl/918/918137b5-a365-4...
3,onbekend,https://signimages.ipsm.nl/091/091e06f3-426f-4...
4,onbekend,https://signimages.ipsm.nl/749/749c45b2-91be-4...
...,...,...
1858917,G11,https://signimages.ipsm.nl/537/537b0b4d-d536-4...
1858918,G11,https://signimages.ipsm.nl/24f/24f97596-448b-4...
1858919,G11,https://signimages.ipsm.nl/440/44047062-2fb3-4...
1858920,G11,https://signimages.ipsm.nl/f84/f8435d28-9ec8-4...


In [21]:
(signs_df['rvv_code'].value_counts()).tolist()

[179894,
 165650,
 100938,
 100918,
 94880,
 82716,
 77395,
 63436,
 63047,
 59506,
 55327,
 55310,
 46987,
 39341,
 36974,
 30884,
 26540,
 26408,
 25523,
 23207,
 20796,
 20665,
 20257,
 18288,
 17915,
 17520,
 16963,
 16138,
 15965,
 15851,
 14263,
 13168,
 12753,
 12723,
 12686,
 12141,
 11270,
 8675,
 8539,
 8461,
 7728,
 7295,
 7147,
 6826,
 5931,
 5812,
 5359,
 5246,
 5131,
 5005,
 4940,
 4747,
 4712,
 4027,
 3953,
 3916,
 3763,
 3751,
 3573,
 3528,
 3527,
 3446,
 3443,
 3373,
 3326,
 3244,
 3216,
 3097,
 3031,
 2967,
 2965,
 2782,
 2601,
 2598,
 2542,
 2252,
 2045,
 1992,
 1853,
 1807,
 1753,
 1704,
 1665,
 1661,
 1651,
 1590,
 1571,
 1566,
 1553,
 1509,
 1467,
 1462,
 1409,
 1406,
 1399,
 1374,
 1364,
 1339,
 1336,
 1334,
 1319,
 1316,
 1178,
 1056,
 946,
 937,
 863,
 862,
 712,
 688,
 658,
 631,
 543,
 535,
 481,
 474,
 470,
 439,
 408,
 380,
 378,
 326,
 279,
 278,
 254,
 244,
 230,
 228,
 217,
 216,
 190,
 188,
 187,
 187,
 173,
 160,
 158,
 143,
 137,
 136,
 127,
 126,
 11

In [197]:
img

,rvv_code,image
0,H1,https://signimages.ipsm.nl/513/5131f682-f69b-4...
1,onbekend,https://signimages.ipsm.nl/bbb/bbb282fa-3822-4...
2,onbekend,https://signimages.ipsm.nl/918/918137b5-a365-4...
3,onbekend,https://signimages.ipsm.nl/091/091e06f3-426f-4...
4,onbekend,https://signimages.ipsm.nl/749/749c45b2-91be-4...
...,...,...
1858917,G11,https://signimages.ipsm.nl/537/537b0b4d-d536-4...
1858918,G11,https://signimages.ipsm.nl/24f/24f97596-448b-4...
1858919,G11,https://signimages.ipsm.nl/440/44047062-2fb3-4...
1858920,G11,https://signimages.ipsm.nl/f84/f8435d28-9ec8-4...


In [29]:
img['rvv_code'].unique()

array(['H1', 'onbekend', 'J27', 'A1', 'E6', 'E1', 'G13', 'E8', 'G6',
       'A2(Zone)', 'G12a', 'C12', 'C1', 'B1', 'D2', 'G7(Zone)', 'C4',
       'C7', 'A1(Zone)', 'C20', 'E4', 'D4', 'C19', 'B3', 'E1(Zone)', 'C9',
       'K14', 'F6', 'A4', 'C2', 'J21', 'C3', 'B6', 'J5', 'F5', 'G11',
       'J38', 'G5', 'J4', 'A3', 'G7', 'G14', 'A2', 'E7', 'L8', 'C15',
       'E10(Zone)', 'J23', 'C22a', 'D5', 'C6', 'E2', 'L2', 'H2', 'J22',
       'J24', 'D6', 'J37', 'B5', 'L1', 'J16', 'G12b', 'G12', 'D3', 'C16',
       'J10', 'J9', 'J29', 'J32', 'J8', 'B4', 'F13', 'G3', 'B2', 'D1',
       'C17', 'E9', 'E9(Zone)', 'C8', 'C7(Zone)', 'J2', 'E3', 'J13',
       'J17', 'F1', 'J1', 'G2', 'J11', 'B7', 'L14', 'J12', 'G4', 'J3',
       'F8', 'C21', 'L19', 'E11', 'J19', 'J18', 'G1', 'E5', 'G9', 'C14',
       'J15', 'J20', 'C10', 'J28', 'C13', 'F2', 'D7', 'F7', 'E12', 'F21',
       'E3(Zone)', 'F4', 'J39', 'J6', 'J25', 'F10', 'A5', 'G8', 'J36',
       'C5', 'C18', 'L13', 'E13', 'C7b', 'C22', 'L10', 'F3', 'J31', 'L3

Here dropping some traffic sign codes due to not having enough modeling images

In [148]:
drop_rows = img_copy[img_copy['rvv_code'].isin(['J34', 'C22b', 'C23-03', 'L11', 'L3c', 'F16', 'C23-02', 'L3a', 'C23-01', 'L18', 'L6', 'L7', 'F18','J35', 'A1E1(Zone)','F11', 'F12', 'F14', 'L15', 'F22', 'onbekend'])].index
drop_rows

Int64Index([      1,       2,       3,       4,       5,       7,      11,
                 12,      14,      15,
            ...
            1858601, 1858640, 1858755, 1858759, 1858762, 1858780, 1858793,
            1858816, 1858857, 1858858],
           dtype='int64', length=166451)

In [150]:
drop_rvv= img[img['rvv_code'].isin(['J34', 'C22b', 'C23-03', 'L11', 'L3c', 'F16', 'C23-02', 'L3a', 'C23-01', 'L18', 'L6', 'L7', 'F18','J35', 'A1E1(Zone)','F11', 'F12', 'F14', 'L15', 'F22', 'onbekend'])].index
drop_rvv

Int64Index([      1,       2,       3,       4,       5,       7,      11,
                 12,      14,      15,
            ...
            1858601, 1858640, 1858755, 1858759, 1858762, 1858780, 1858793,
            1858816, 1858857, 1858858],
           dtype='int64', length=166451)

In [151]:
img = img.drop(drop_rows,axis=0)
img

,rvv_code,image
0,H1,https://signimages.ipsm.nl/513/5131f682-f69b-4...
6,J27,https://signimages.ipsm.nl/5d0/5d018fa1-1541-4...
8,A1,https://signimages.ipsm.nl/999/999f91b1-4848-4...
9,A1,https://signimages.ipsm.nl/887/8876b419-f22b-4...
10,E6,https://signimages.ipsm.nl/e72/e720084e-34b6-4...
...,...,...
1858917,G11,https://signimages.ipsm.nl/537/537b0b4d-d536-4...
1858918,G11,https://signimages.ipsm.nl/24f/24f97596-448b-4...
1858919,G11,https://signimages.ipsm.nl/440/44047062-2fb3-4...
1858920,G11,https://signimages.ipsm.nl/f84/f8435d28-9ec8-4...


In [152]:
img.reset_index(drop=True)

,rvv_code,image
0,H1,https://signimages.ipsm.nl/513/5131f682-f69b-4...
1,J27,https://signimages.ipsm.nl/5d0/5d018fa1-1541-4...
2,A1,https://signimages.ipsm.nl/999/999f91b1-4848-4...
3,A1,https://signimages.ipsm.nl/887/8876b419-f22b-4...
4,E6,https://signimages.ipsm.nl/e72/e720084e-34b6-4...
...,...,...
1692466,G11,https://signimages.ipsm.nl/537/537b0b4d-d536-4...
1692467,G11,https://signimages.ipsm.nl/24f/24f97596-448b-4...
1692468,G11,https://signimages.ipsm.nl/440/44047062-2fb3-4...
1692469,G11,https://signimages.ipsm.nl/f84/f8435d28-9ec8-4...


In [181]:
img['rvv_code'].unique()

array(['H1', 'J27', 'A1', 'E6', 'E1', 'G13', 'E8', 'G6', 'A2(Zone)',
       'G12a', 'C12', 'C1', 'B1', 'D2', 'G7(Zone)', 'C4', 'C7',
       'A1(Zone)', 'C20', 'E4', 'D4', 'C19', 'B3', 'E1(Zone)', 'C9',
       'K14', 'F6', 'A4', 'C2', 'J21', 'C3', 'B6', 'J5', 'F5', 'G11',
       'J38', 'G5', 'J4', 'A3', 'G7', 'G14', 'A2', 'E7', 'L8', 'C15',
       'E10(Zone)', 'J23', 'C22a', 'D5', 'C6', 'E2', 'L2', 'H2', 'J22',
       'J24', 'D6', 'J37', 'B5', 'L1', 'J16', 'G12b', 'G12', 'D3', 'C16',
       'J10', 'J9', 'J29', 'J32', 'J8', 'B4', 'F13', 'G3', 'B2', 'D1',
       'C17', 'E9', 'E9(Zone)', 'C8', 'C7(Zone)', 'J2', 'E3', 'J13',
       'J17', 'F1', 'J1', 'G2', 'J11', 'B7', 'L14', 'J12', 'G4', 'J3',
       'F8', 'C21', 'L19', 'E11', 'J19', 'J18', 'G1', 'E5', 'G9', 'C14',
       'J15', 'J20', 'C10', 'J28', 'C13', 'F2', 'D7', 'F7', 'E12', 'F21',
       'E3(Zone)', 'F4', 'J39', 'J6', 'J25', 'F10', 'A5', 'G8', 'J36',
       'C5', 'C18', 'L13', 'E13', 'C7b', 'C22', 'L10', 'F3', 'J31', 'L3b',
       '

In [69]:
val = pd.DataFrame(img['rvv_code'].value_counts()[img['rvv_code'].unique()]).reset_index()
val.tail()

,index,rvv_code
165,C23-01,5
166,L18,10
167,L6,278
168,L7,228
169,F18,3


Choosing the first 500 images for our training dataset

In [290]:
url_list = []
# i = 0
for i in range(len(img['rvv_code'].value_counts())):
    url = (img[(img['rvv_code'].isin([img['rvv_code'].unique()[i]]))]['image']).head(500)
    url_list.append(url)
#     i+=1
    

In [189]:
#it's commented out because of the long list
# url_list

In [15]:
def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

In [292]:
url_list_flattened = flatten(url_list)

In [293]:
len(url_list_flattened)

65150

to save our images to a folder

In [295]:
%%time
image = 0

for link in url_list_flattened:

    response = requests.get(link)
    file = open(f'images/image_{str(image)}.jpg', 'wb')
    file.write(response.content)
    file.close()

    image += 1

Wall time: 1h 45min 18s


now time to get our test images from the long list

In [153]:
%%time
url_list_test = []
# i = 0
for i in range(len(img['rvv_code'].value_counts())):
    url_test = (img[(img['rvv_code'].isin([img['rvv_code'].unique()[i]]))]['image']).tail(100)
    url_list_test.append(url_test)
#     i+=1
    

Wall time: 1min 14s


In [154]:
url_list_test_flattened = flatten(url_list_test)

In [162]:
url_list_test_flattened

['https://signimages.ipsm.nl/2d6/2d6d5f6f-6f61-4461-a97d-f4bd05f6c085.jpg',
 'https://signimages.ipsm.nl/e75/e754124e-0c00-4826-9f70-51d229f834fa.jpg',
 'https://signimages.ipsm.nl/d25/d252264b-c89c-48ea-a172-9fa7f6b5df6d.jpg',
 'https://signimages.ipsm.nl/1e5/1e54e1e3-832a-44fb-adc7-762d9da475a2.jpg',
 'https://signimages.ipsm.nl/1d1/1d1127b2-8253-4d9d-baa3-af4339b93b83.jpg',
 'https://signimages.ipsm.nl/740/740f63fd-3054-4048-ac5e-0329ed6bdb5b.jpg',
 'https://signimages.ipsm.nl/864/86494994-5236-4850-be64-68ee52a198ea.jpg',
 'https://signimages.ipsm.nl/95a/95ab0a04-0741-458f-bfe0-23c465c9dbda.jpg',
 'https://signimages.ipsm.nl/979/9792307e-dce9-4f73-8c96-70fc91481cb3.jpg',
 'https://signimages.ipsm.nl/00a/00ad648f-05e7-4b15-bca0-4277ad726960.jpg',
 'https://signimages.ipsm.nl/f03/f03dbb86-23ba-47da-983b-dd5c1a80c18d.jpg',
 'https://signimages.ipsm.nl/d96/d96f3350-6b9a-4656-9976-0a9f18f91f82.jpg',
 'https://signimages.ipsm.nl/b3a/b3a9ce76-81bf-428e-a4d7-e8f33a355681.jpg',
 'https://si

In [175]:
%%time
rvv_code_test = []
# i = 0
for i in range(len(img['rvv_code'].value_counts())):
    rvv_code = (img[(img['rvv_code'].isin([img['rvv_code'].unique()[i]]))]['rvv_code']).tail(15)
    rvv_code_test.append(rvv_code)
#     i+=1
    

Wall time: 1min 24s


In [176]:
rvv_code_test_flattened = flatten(rvv_code_test)

In [178]:
%%time
image = 0o000

for link in url_list_test_flattened:

    response = requests.get(link)
    file = open(f'images_test_2/image_test_{str(image)}.jpg', 'wb')
    file.write(response.content)
    file.close()

    image += 1

Wall time: 23min 39s


here we are building another dataframe to use in our validation part in notebook 'traffic_classifier'

In [177]:
test_df = pd.DataFrame(rvv_code_test_flattened, columns=['ClassId'])
test_df

,ClassId
0,H1
1,H1
2,H1
3,H1
4,H1
...,...
2229,J7
2230,J7
2231,J7
2232,J7


In [172]:
l_df = pd.read_csv('l_df.csv')
l_df=l_df.rename(columns={'0':'Path'})
l_df

,Path
0,image_test_0.jpg
1,image_test_1.jpg
2,image_test_2.jpg
3,image_test_3.jpg
4,image_test_4.jpg
...,...
14546,image_test_14546.jpg
14547,image_test_14547.jpg
14548,image_test_14548.jpg
14549,image_test_14549.jpg


In [173]:
test_df['Path'] = l_df['Path']
test_df

,ClassId,Path
0,H1,image_test_0.jpg
1,H1,image_test_1.jpg
2,H1,image_test_2.jpg
3,H1,image_test_3.jpg
4,H1,image_test_4.jpg
...,...,...
14546,J7,image_test_14546.jpg
14547,J7,image_test_14547.jpg
14548,J7,image_test_14548.jpg
14549,J7,image_test_14549.jpg


In [174]:
test_df.to_csv('Test_df.csv', index=False)

In [160]:
len(url_list_test_flattened)

14551

In [54]:
signs_df['rvv_code'][0]

'H1'

In [7]:
signs_df.to_csv('traffic_signs.csv', index=False)